## Instruction

1. Collect all type II cyclases (HMMER: https://www.ebi.ac.uk/Tools/hmmer/search/hmmsearch)
2. Create sequence similarity network (https://efi.igb.illinois.edu/efi-est/)
3. Create genome neighborhood network (https://efi.igb.illinois.edu/efi-gnt/)
4. Download Sqlite database
5. Export "attribute" table to csv (type II cyclase info)
6. Export "neighbor" table to csv (neighbor info)

## Preamble

In [ ]:
import pandas as pd
import re
import numpy as np
import subprocess
import glob
from subprocess import Popen, PIPE
import os

from Bio import ExPASy
from Bio import SwissProt
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
#from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio import Entrez

## Functions

In [ ]:
def add_rel_pos(cyclase_table, neighbor_table, neighbor_output, cyclase_output):
    '''
    input:
    cyclase_table: str, path for the csv file with cyclase info (attributes in sqlite)
    neighbor_table: str, path for the csv file with neighboring genes info (neighbor in sqlite)
    
    output:
    neighbor_output: str, path for csv file with neighbor + relative position
    cyclase_output: str, path for updated cyclase file (- no neighbor cyclase / + gene_key)
    '''
    ### num column in both dataframe refers to the location number of the gene in a contig
    neighbor_df = pd.read_csv(neighbor_table)
    typeii_df = pd.read_csv(cyclase_table)

    ### additional column for gene position relative to the type II cyclase gene
    neighbor_df['rel_pos'] = 0


    ### find the location number of ith type II cyclase
    ### substract the typeII cyclase location to get the relative position for each gene
    ### gene_key in neighbor file = sort_key in cyclase file
    for ind, val in typeii_df.iterrows():
        typeii_pos = val.num
        neighbor_df.loc[neighbor_df['gene_key'] == val.sort_key, 'rel_pos'] = neighbor_df.loc[neighbor_df['gene_key'] == val.sort_key, 'num'] - typeii_pos

        
    neighbor_df.to_csv(neighbor_output, index = False)
    print('...done adding relative positions to cyclases')
    
    
    with_neighbor = set(neighbor_df.gene_key)
    typeii_df['gene_key'] = typeii_df.sort_key
    typeii_df.loc[typeii_df['gene_key'].isin(with_neighbor)].to_csv(cyclase_output, index = False)
    print('...done updating the cyclase file (remove no neighbor cyclases)')

In [ ]:
def filter_cluster(neighbor_file, cyclase_file, window, interpro, not_interpro, neighbor_out, cyclase_out, selection, output = False):
    '''
    input:
    neighbor_file: str, path for the neighbor csv file with relative position
    cyclase_file: str, path for the cyclase csv file
    window: int, window around typeII cyclase to inspect
    interpro: list, protein family to include
    not_interpro: list, protein family to exclude
    neighbor_out: str, path for the csv file name after filtering
    cyclase_out: str, path for the csv file name after filtering
    selection: str, name for this selection
    output: Bool, save the output or not
       
    output: selected cyclase_file and neighbor_file
    '''

    neighbor_df = pd.read_csv(neighbor_file, index_col = 'sort_key')
    potential_cluster = set()

    ### extract dataframe within the window from typeII_cyclase
    sele = neighbor_df.loc[np.abs(neighbor_df['rel_pos']) <= window]

    interpro_match = sele.loc[(sele['ipro_family'].str.contains('|'.join(interpro)) == True)&
                                (sele['ipro_family'].str.contains('|'.join(not_interpro)) == False)]
       
    potential_cluster = list(set(interpro_match.gene_key))
    print(f'{cyclase_file}: selection based on {selection}, window {str(window)}: {str(len(potential_cluster))} loci')

    if output:
        typeii_df = pd.read_csv(cyclase_file, index_col = 'sort_key')
        typeii_df.loc[typeii_df['gene_key'].isin(potential_cluster)].to_csv(cyclase_out)
        
        neighbor_df.loc[neighbor_df['gene_key'].isin(potential_cluster)].to_csv(neighbor_out)

In [ ]:
def select_P450(neighbor_table, distance, outfile = ''):
    '''
       neighbor_table: str, path for the neighbor csv file after selection
       distance: int, how far a P450 from type II cyclase will be included
       outfile: str, path for the output csv file. If outfile = '', do not save output
    '''
    
    neighbor = pd.read_csv(neighbor_table)
    p450 = neighbor.loc[(neighbor['ipro_family'].str.contains('IPR036396')) 
                           & (np.abs(neighbor['rel_pos']) <= distance)]
    
    print(f'{str(len(p450))} P450s were found at distance {str(distance)}')
    
    if outfile != '':
        p450['accession'].to_csv(f'P450_list.txt', index = False) ## for SSN
        p450[['sort_key','accession','id','seq_len','gene_key','rel_pos','cyclase_accession','cluster_num','organism']].to_csv(
             outfile, index = False) 

## Processing data

#### 1. add relative position of each neighboring genes to the type II cyclases

In [ ]:
add_rel_pos('raw/cyclase_GNN_attribute.csv',    ### csv with type II cyclase info
            'raw/neighbor_GNN.csv',             ### csv with neighboring genes info
            'neighbor_pos.csv',                 ### output file: neighboring genes + relative position to cyclase
            'cyclase_key.csv')                  ### output file: cyclase + updated gene_key

#### 2. Find genomic loci with type I cyclase close to type II cyclases

In [ ]:
typeI_interpro = ['IPR008949', 'IPR034686', 'IPR000537', 'IPR036424', 'IPR005630']
poly_interpro = ['IPR000092']  ## polyprenyl synthases have both IPR008949/IPR000092

filter_cluster(neighbor_file = 'neighbor_pos.csv',                # neighbor csv file with relative position
               cyclase_file = 'cyclase_key.csv',                  # cyclase csv file
               window = 5,                                        # inspection window
               interpro = typeI_interpro,                         # protein family to include
               not_interpro = poly_interpro,                      # protein family to exclude
               neighbor_out = 'neighbor_sele_typeI.csv',          # output neighbor file
               cyclase_out = 'cyclase_sele_typeI.csv',            # output cyclase file
               selection = 'typeI',                               # selection name
               output = True)                                     # output the result or not

#### 3. Find genomic loci with hopanoid biosynthetic genes

In [ ]:
hopanoid_interpro = ['IPR017827', 'IPR002060', 'IPR033904', 'IPR017828', 'IPR019845', 'IPR017830']

filter_cluster(neighbor_file = 'neighbor_sele_typeI.csv', # neighbor csv file after type I selection
               cyclase_file = 'cyclase_sele_typeI.csv',   # cyclase csv file after type I selection
               window = 10,                               # inspection window
               interpro = hopanoid_interpro,              # protein family to include
               not_interpro = ['IPR999999'],              # protein family to exclude (IPR999999 is a decoy)
               neighbor_out = 'neighbor_sele_hop.csv',  # output neighbor file
               cyclase_out = 'cyclase_sele_hop.csv',    # output cyclase file
               selection = 'hopanoid',                    # selection name
               output = True)                             # output the result or not

#### 4. loci from 2. - loci from 3.

In [ ]:
hop_key = pd.read_csv('cyclase_sele_hop.csv').gene_key

cyclase = pd.read_csv('cyclase_sele_typeI.csv', index_col = 'sort_key')
neighbor = pd.read_csv('neighbor_sele_typeI.csv', index_col = 'sort_key')

cyclase_final = cyclase.loc[(cyclase['gene_key'].isin(hop_key)) == False]

## Add cyclase accession number and cluster number (in the network file) to the neighbors
   
for ind, val in cyclase_final.iterrows():
    neighbor_final.loc[neighbor_final.gene_key == val.gene_key, 'cyclase_accession'] = val.accession
    neighbor_final.loc[neighbor_final.gene_key == val.gene_key, 'cluster_num'] = val.cluster_num
    neighbor_final.loc[neighbor_final.gene_key == val.gene_key, 'organism'] = val.organism
    

neighbor_final = neighbor_final.loc[(neighbor['gene_key'].isin(hop_key)) == False]
cyclase_final.to_csv('cyclase_sele_final.csv')
neighbor_final.to_csv('neighbor_sele_final.csv')

print(f'final selection has {len(cyclase_final)} loci (gene clusters)')

#### 5. Find P450s from the loci identified in 4.

In [ ]:
select_P450('neighbor_sele_final.csv', 10, outfile = 'P450_final2.csv')